In [1]:
import requests
import time
from datetime import datetime
import pytz  # For timezone handling
import csv


import folium
from folium.plugins import HeatMap
import pandas as pd


In [2]:


# Replace with your actual Blynk Auth Tokens
token_lat = "wm9lYKJECPhtQuZMPjQWCZtR8DNHF1aA"
token_lon = "wm9lYKJECPhtQuZMPjQWCZtR8DNHF1aA"
token_val = "C1RPiDbUrLG3sUY2JhOMcpkj4SxIfnB8"

# API endpoints
url_lat = f"https://blynk.cloud/external/api/get?token={token_lat}&v1"
url_lon = f"https://blynk.cloud/external/api/get?token={token_lon}&v2"
url_val = f"https://blynk.cloud/external/api/get?token={token_val}&v2"

# Create or open a CSV file named "data.csv"
filename = "data.csv"
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Datetime", "Latitude", "Longitude", "Value"])

    while True:
        try:
            # Fetch latitude, longitude, and value
            response_lat = requests.get(url_lat)
            response_lon = requests.get(url_lon)
            response_val = requests.get(url_val)

            if response_lat.status_code == 200 and response_lon.status_code == 200 and response_val.status_code == 200:
                data_lat = response_lat.json()
                data_lon = response_lon.json()
                data_val = response_val.json()

                # Get the current date and time in Bangladesh time zone
                bangladesh_tz = pytz.timezone('Asia/Dhaka')
                current_time = datetime.now(bangladesh_tz).strftime("%Y-%m-%d %H:%M:%S")

                # Extract values
                lat = data_lat if isinstance(data_lat, float) else "Invalid Latitude"
                lon = data_lon if isinstance(data_lon, float) else "Invalid Longitude"
                value = data_val if isinstance(data_val, (float, int)) else "Invalid Value"

                # Write the current time, latitude, longitude, and value to the CSV file
                writer.writerow([current_time, lat, lon, value])
                print(f"[{current_time}] Latitude: {lat}, Longitude: {lon}, Value: {value}")
            else:
                current_time = datetime.now(pytz.timezone('Asia/Dhaka')).strftime("%Y-%m-%d %H:%M:%S")
                print(f"[{current_time}] Error: One or more API requests failed")

        except requests.RequestException as e:
            current_time = datetime.now(pytz.timezone('Asia/Dhaka')).strftime("%Y-%m-%d %H:%M:%S")
            print(f"[{current_time}] Request Error: {e}")

        time.sleep(1)  # Pause for 1 second


[2024-08-24 22:35:40] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:35:43] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:35:47] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:35:50] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:35:53] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:35:56] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:35:59] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:36:02] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:36:05] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:36:08] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:36:11] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:36:14] Latitude: 23.899895, Longitude: 90.419258, Value: 101.0
[2024-08-24 22:36:17] Latitude: 23.899895, Longitude: 90.419258,

KeyboardInterrupt: 

In [4]:

# Load the data
df = pd.read_csv('data.csv')

# Preview the data
df.head()

# Step 4: Create a Base Map with Black Background
# Get the mean latitude and longitude for centering the map
center_lat = df['Latitude'].mean()
center_long = df['Longitude'].mean()

# Create a base map with the CartoDB Dark Matter tiles
m = folium.Map(location=[center_lat, center_long], zoom_start=10, tiles='CartoDB dark_matter')

# Step 5: Create the Heatmap
# Prepare the data for the heatmap
heat_data = [[row['Latitude'], row['Longitude'], row['Value']] for index, row in df.iterrows()]

# Add the heatmap to the map
HeatMap(heat_data, min_opacity=0.4, max_val=df['Value'].max(), radius=25, blur=15).add_to(m)

# Step 6: Add CircleMarkers with Tooltips
for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,  # You can adjust the radius
        color='black',
        fill=False,
        fill_opacity=0.1,
        tooltip=f"PM2.5: {row['Value']} ppm<br>DateTime: {row['Datetime']}"
    ).add_to(m)

# Step 7: Add a Linear Scale Color Legend Matching the Heatmap Colors
# Define the color gradient for the legend
colors = ['#1abc9c', '#2ecc71', '#f1c40f', '#e67e22', '#e74c3c']

legend_html = '''
<link href="https://fonts.googleapis.com/css2?family=Tomorrow:wght@100&display=swap" rel="stylesheet">
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 200px; height: 60px;
            z-index:9999; font-size:14px; font-family: 'Tomorrow', sans-serif;
            color: white;">
     <b>PM2.5 Concentration</b><br>
     <div style="display: flex;">
         <div style="background-color: {0}; width: 20%; height: 20px;"></div>
         <div style="background-color: {1}; width: 20%; height: 20px;"></div>
         <div style="background-color: {2}; width: 20%; height: 20px;"></div>
         <div style="background-color: {3}; width: 20%; height: 20px;"></div>
         <div style="background-color: {4}; width: 20%; height: 20px;"></div>
     </div>
     <div style="display: flex; justify-content: space-between;">
         <span>Low</span><span>High</span>
     </div>
</div>
'''.format(*colors)

m.get_root().html.add_child(folium.Element(legend_html))

# Step 8: Add a Banner with "STARDUST" on the Left (White, Bold, Italic) and Datetime Range on the Right
# Determine the datetime range
datetime_from = df['Datetime'].min()
datetime_to = df['Datetime'].max()

banner_html = f'''
<link href="https://fonts.googleapis.com/css2?family=Tomorrow:wght@400&display=swap" rel="stylesheet">
<div style="position: fixed;
            top: 0; left: 0; width: 100%; height: 10%;
            background-color: #2ecc71; z-index:9999; font-family: 'Tomorrow', sans-serif;
            color: black; display: flex; align-items: center; justify-content: space-between;
            padding: 10px 20px;">
    <div style="font-size: 24px; font-weight: bold; font-style: italic; color: white;">
        STARDUST
    </div>
    <div style="text-align: right; font-size: 14px; font-weight: 400;">
        {datetime_from} to <br> {datetime_to}
    </div>
</div>
'''

m.get_root().html.add_child(folium.Element(banner_html))

# Step 9: Display the Map
m

# Step 10: Save the Map as an HTML File
#m.save('pm5_heatmap_with_tooltips_and_linear_scale_legend_with_banner.html')



<ipython-input-4-8321970beb36>:20: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data, min_opacity=0.4, max_val=df['Value'].max(), radius=25, blur=15).add_to(m)
